In [1]:
%load_ext autoreload
%autoreload 2
%autoreload 2

In [21]:
from degen_sim.common.notebook_utils import hide_raw_cells, markdown

In [22]:
hide_raw_cells()

In [23]:
import numpy as np
import pandas as pd
import plotly.express as px

from datetime import datetime
from dateutil.tz import gettz
import os
import time

from dataclasses import dataclass, asdict

In [24]:
from degen_sim.common.constants import ROOT_DIR, DATA_DIR

In [25]:
def line_scatter(df, x, y, **kwargs):
    fig = px.line(df, x=x, y=y, **kwargs, template="plotly_white")
    for data in fig.data:
        data.update(mode="markers+lines")
    return fig

In [26]:
picks = pd.read_csv(os.path.join(DATA_DIR, "parlay_tracker_nfl.csv"))

In [27]:
picks = picks[~picks.Win.isna()]

In [28]:
pickers = sorted(picks["Pick"].unique())

In [29]:
@dataclass
class PickInfo:
    """
    Represents a product with a name, price, and quantity in stock.
    """
    name: str
    num_wins: int
    num_losses: int
    num_pushes: int
    odds: list[float]

In [30]:
def american_to_implied_prob(odds):
    """
    Convert American odds to implied probability.

    Parameters:
        odds (int or float): American odds (e.g., +150, -120)

    Returns:
        float: Implied probability as a percentage (e.g., 40.0 for +150)
    """
    if odds > 0:
        prob = 100 / (odds + 100)
    elif odds < 0:
        prob = abs(odds) / (abs(odds) + 100)
    else:
        raise ValueError("Odds cannot be zero")

    return prob


In [31]:
pick_infos = []
for picker in pickers:
    sub_df = picks[picks.Pick == picker].copy()
    odds = [american_to_implied_prob(odds) for odds in sub_df["Odds"]]
    num_wins = len(sub_df[sub_df["Win"] == "Y"])
    num_losses = len(sub_df[sub_df["Win"] == "N"])
    num_pushes = len(sub_df[sub_df["Win"] == "P"])
    assert num_wins + num_losses + num_pushes == len(sub_df)
    pick_info = PickInfo(name=picker, num_wins=num_wins, num_losses=num_losses, num_pushes=num_pushes, odds=odds)
    pick_infos.append(pick_info)

In [32]:
def simulate_pick_info(pick_info, num_trials=100, seed=0):
    np.random.seed(seed)
    
    num_games = pick_info.num_wins + pick_info.num_losses + pick_info.num_pushes
    df = pd.DataFrame({"game_index": range(num_games * num_trials), "p": np.random.uniform(low=0.0, high=1.0, size=num_games * num_trials)})
    df["trial_index"] = df["game_index"] // num_games
    df["implied_prob"] = pick_info.odds * num_trials
    df["win"] = np.where(df["p"] <= df["implied_prob"], 1.0, 0.0)

    total_wins = df.groupby("trial_index")[["win"]].sum().reset_index()
    return total_wins

In [33]:
cdfs = []
for pick_info in pick_infos:
    result = simulate_pick_info(pick_info, num_trials=1_000_000)
    num_wins = pick_info.num_wins
    cdf = len(result[result.win <= num_wins]) / len(result)
    cdfs.append(cdf)

In [34]:
cdf_df = pd.DataFrame([asdict(pick_info) for pick_info in pick_infos])
cdf_df["cdf"] = cdfs

In [35]:
cdf_df.sort_values(by="cdf", ascending=False).drop(columns="odds")

,name,num_wins,num_losses,num_pushes,cdf
1,Arun,3,1,0,0.924610
6,Kunal,3,1,0,0.918674
0,Ananya,2,2,0,0.672756
2,Ben,2,2,0,0.646160
9,Ryan,2,2,0,0.592147
4,Cristian,2,2,0,0.508063
3,Caleb,1,3,0,0.286183
5,Daniel,1,3,0,0.239734
8,Michael,1,3,0,0.227499
7,Kyle,1,3,0,0.201340
